# Facteur de Forme dans une scène test

Maintenant qu'on a une idée de comment sont calculés les facteurs de forme et les effets de la méthode, on va pouvoir tester ça sur une mini scène.

## TODO

- graph experimental (rnd et uniforme) vs théorique pour les patchs où le calcul est possible
- graph temps rnd vs uniforme avec différentes valeurs de n et dx,dy,dz

## La scène

- une pièce de 3x3 complètement close avec un sol, des murs et un plafond
- entièrement vide pour éviter d'avoir à se soucier des occlusions
- la représentation se base sur des surfaces rectangulaires
    - une cellule au sol et au plafond fait 1x1
    - une cellule des murs 1x2

## Imports

In [1]:
import numpy as np
import random

print("imports: OK")

rng = random.SystemRandom()
print("random generator: OK")

imports: OK
random generator: OK


## Rectangle

- représente la surface d'un élément de la scène.
- aligné par rapport à deux axes (dans le plan xOy, xOz ou y0z)
- identifié par un numéro (qui servira d'index pour le maillage)
- définit par 2 points cmin, cmax et un vecteur normale à la surface
- permet d'obtenir un point tiré aléatoirement sur la surface du rectangle
- permet de subdiviser la surface en carrés de même taille

**attention:** par défaut, les surfaces sont divisées en carré de 0.5 unité par coté

In [16]:
class Rect():
    count = 0

    def __init__(self, cmin, cmax, norm):
        # Index
        self.idt = Rect.count
        Rect.count += 1
        
        # Points and normal
        self.cmin = cmin
        self.cmax = cmax
        self.norm = norm
        
        # Length
        self.lx = cmax[0] - cmin[0]
        self.ly = cmax[1] - cmin[1]
        self.lz = cmax[2] - cmin[2]
        
        # Area
        self.area = self.lx*self.ly + self.lx*self.lz + self.ly*self.lz
        
        # Split surface into patchs
        self.split()
        print(f"{self.idt} {self.cmin} {self.cmax} area:{self.area}")

    def getRndUV(self, umin, umax, vmin, vmax):
        # TODO: p have to be in ]0;1[ to avoid the r = 0 case (in Ri T Rj)
        pu = rng.uniform(umin, umax)
        pv = rng.uniform(vmin, vmax)
        return (pu, pv)
    
    def getRandomPoint(self):
        if (self.lz == 0):
            # R xOy
            (px, py) = self.getRndUV(self.cmin[0], self.cmax[0], self.cmin[1], self.cmax[1])
            pz = self.cmin[2]
        elif (self.ly == 0):
            # R xOz
            (px, pz) = self.getRndUV(self.cmin[0], self.cmax[0], self.cmin[2], self.cmax[2])
            py = self.cmin[1]
        else:
            # R yOz
            (py, pz) = self.getRndUV(self.cmin[1], self.cmax[1], self.cmin[2], self.cmax[2])
            px = self.cmin[0]
        #print(f"(idt:{self.idt} return point ({px}, {py}, {pz})");
        return(np.array([px, py, pz]))
    

    def getPatchs(self):
        return self.patch

    def split(self, dx=0.5, dy=0.5, dz=0.5):
        self.patch = []

        # Hack to ensure we do at least 1 iteration
        lx = dx  if self.lx == 0 else self.lx
        ly = dy  if self.ly == 0 else self.ly
        lz = dz  if self.lz == 0 else self.lz

        # used to cancel splits on u, v, w
        kx = 0. if lx < 1. else 1.
        ky = 0. if ly < 1. else 1.
        kz = 0. if lz < 1. else 1.

        x0 = self.cmin[0]
        y0 = self.cmin[1]
        z0 = self.cmin[2]

        z = dz / 2.
        while z < lz:
            y = dy / 2.
            while y < ly:
                x = dx / 2.
                while x < lx:
                    #print(f"{x0+x*kx}, {y0+y*ky}, {z0+z*kz}")
                    self.patch.append(np.array([x0+x*kx, y0+y*ky, z0+z*kz]))
                    x += dx
                y += dy
            z += dz

    def initFormFactors(self, n):
        self.fij = np.zeros([n])

    def dumpFij(self):
        n = len(self.fij)
        for i in range(n):
            print(f"[{self.idt} -> {i}] Fij = {self.fij[i]}")
            
print("class Rectangle: OK")

class Rectangle: OK


## Maillage

Va nous permettre de stocker les surfaces composant la scène. Chaque polygone est identifié par son numéro d'index.

**attention:** c'est une approche brute de fonderie, juste pour les besoins du test !

- contient la liste des polygones qui composent la scène
- permet d'ajouter un polygone à la scène courante
- permet d'accéder à un polygone en connaissant sont numéro
- permet de lancer le calcul des facteurs de forme sur l'ensemble des éléments du maillage (2 versions : aléatoire ou uniforme cellules carrés)
- permet de lancer le calcul du facteur de forme entre 2 élément du maillage
- permet d'afficher les facteurs de forme entre un éléments et tous les autres de la scène

In [6]:
class Mesh():
    def __init__(self):
        self.poly = []

    def add(self, item):
        self.poly.append(item)

    def get(self, index):
        return self.poly[index]

    def initFormFactors(self):
        size = len(self.poly)
        for p in self.poly:
            p.initFormFactors(size)
            

    def getFormFactors_rnd(self, n):
        for poly_i in self.poly:
            for poly_j in self.poly:
                if poly_i.idt != poly_j.idt:
                    poly_i.fij[poly_j.idt] = self.getFormFactor_rnd(poly_i, poly_j, n)
                    
                    

    def getFormFactors_uni_sq(self):
        for poly_i in self.poly:
            for poly_j in self.poly:
                if poly_i.idt != poly_j.idt:
                    poly_i.fij[poly_j.idt] = self.getFormFactor_uni_sq(poly_i, poly_j)

    # get Fij with uniform sampling (square patchs of the same size)
    def getFormFactor_uni_sq(self, ri, rj):
        fij = 0.
        aj = rj.area
        patchs_i = ri.getPatchs()
        patchs_j = rj.getPatchs()
        li = len(patchs_i)
        lj = len(patchs_j)
        samples = li * lj

        for xi in patchs_i:
            for xj in patchs_j:
                # get ray from r1 to r2
                rij = xj - xi
                r = np.linalg.norm(rij)
                nrij = rij / r
                r2 = r*r
                costhetai = np.dot(ri.norm, nrij)
                costhetaj = -1.*np.dot(rj.norm, nrij)
                deltaf = ((costhetai * costhetaj) / (np.pi * r2 + aj/lj))
                if (deltaf > 0.):
                    fij += deltaf
        fij = fij * aj/samples
        return fij

    # get Fij with random sampling (ray casting Emitter x Receiver)
    def getFormFactor_rnd(self, ri, rj, n=1):
        fij = 0.
        aj = rj.area
        for i in range(n):
            # get a point on r1 and r2
            xi = ri.getRandomPoint()
            xj = rj.getRandomPoint()
            #print(f"xi: {xi} xj: {xj}")
            # get ray from r1 to r2
            rij = xj - xi
            r = np.linalg.norm(rij)
            nrij = rij / r
            #print(f"normalized ray vector: {nrij} ({nrij[0]*nrij[0] + nrij[1]*nrij[1] + nrij[2]*nrij[2]})")
            r2 = r*r
            costhetai = np.dot(ri.norm, nrij)
            costhetaj = -1.*np.dot(rj.norm, nrij)
            #print(f"costhetai: {costhetai}, costhetaj: {costhetaj} r2: {r2}")
            deltaf = ((costhetai * costhetaj) / (np.pi * r2 + aj/n))
            #print(f"deltaf: {deltaf}")
            if (deltaf > 0.):
                fij += deltaf
        fij = fij * aj/n
        return fij
    
    def showFij(self, i):
        self.poly[i].dumpFij()

print("class Mesh: OK")

class Mesh: OK


## Quelques valeurs théoriques

__[A CATALOG OF RADIATION HEAT TRANSFER CONFIGURATION FACTORS](http://www.thermalradiation.net/tablecon.html)__ de John R. Howell 

En particulier :

- __[Identical, parallel, directly opposed rectangles](http://www.thermalradiation.net/sectionc/C-11.html)__
- __[Two finite rectangles of same length, having one common edge, and at an angle of 90° to each other](http://www.thermalradiation.net/sectionc/C-14.html)__
- __[Rectangle to rectangle in a parallel plane](http://www.thermalradiation.net/sectionc/C-13.html)__
- __[Rectangle to rectangle in a perpendicular plane; all boundaries are parallel or perpendicular to x and ξ boundaries](http://www.thermalradiation.net/sectionc/C-15.html)__ (**note:** ne fonctionne pas si les 2 rectangles ont un coté sur (Ox))

In [7]:
def getThFij(a, b, c):
    X = a/c
    Y = b/c
    th_fij = 2/(np.pi*X*Y) * ( np.log(np.power((1+X*X)*(1+Y*Y) / (1+X*X+Y*Y), 1/2  ))
        + X*np.sqrt(1+Y*Y)*np.arctan(X/np.sqrt(1+Y*Y))
        + Y*np.sqrt(1+X*X)*np.arctan(Y/np.sqrt(1+X*X))
        - X*np.arctan(X) - Y*np.arctan(Y))
    print(f"theoritical para. ({a},{b},{c}) Fij = {th_fij}")

def getThPerpFij(a, b, c):
    H = a/c
    W = b/c
    th_fij = (1./(np.pi*W)) * (W*np.arctan(1./W) + H*np.arctan(1./H) - np.sqrt(H*H+W*W)*np.arctan(1./np.sqrt(H*H+W*W))
                                + (1./4.) * np.log( ((1+W*W)*(1+H*H) / (1+W*W+H*H))
                                                * np.power((W*W*(1+W*W+H*H) / ((1+W*W)*(W*W+H*H))),W*W)
                                                * np.power((H*H*(1+W*W+H*H) / ((1+H*H)*(H*H+W*W))),H*H)
                                                ))
    print(f"theoritical perp. ({a},{b},{c}) Fij = {th_fij}")

def g_para_z(x, y, eta, xi, z):
    res = (1./(2.*np.pi)) * (
            (y - eta)*np.sqrt( (x-xi)*(x-xi) + z*z ) * np.arctan( (y-eta)/np.sqrt((x-xi)*(x-xi) + z*z ) )
            + (x-xi)*np.sqrt(  (y-eta)*(y-eta) + z*z ) * np.arctan( (x-xi)/np.sqrt((y-eta)*(y-eta) + z*z ) )
            - (z*z/2.)*np.log( (x-xi)*(x-xi) + (y-eta)*(y-eta) + z*z )
            )
    return res

def getThFij_para_z(r1, r2):
    # http://www.thermalradiation.net/sectionc/C-13.html
    z = r2.cmax[2] - r1.cmax[2]
    x = np.array([r1.cmin[0], r1.cmax[0]])
    y = np.array([r1.cmin[1], r1.cmax[1]])
    xi = np.array([r2.cmin[0], r2.cmax[0]])
    eta = np.array([r2.cmin[1], r2.cmax[1]])

    f12 = 0.
    for l in range(1,3):
        for k in range(1,3):
            for j in range(1,3):
                for i in range(1,3):
                    f12 += np.power(-1, i+j+k+l)*g_para_z(x[i-1], y[j-1], eta[k-1], xi[l-1], z)
    f12 *= 1./ ( (x[2-1] - x[1-1])*(y[2-1] - y[1-1]) )
    print(f"Th Fij ({r1.idt} -> {r2.idt}) = {f12}")

def g_perp_x(x, y, eta, xi):
    K = (y -eta) / np.sqrt(x*x + xi*xi)
    res = (1./(2.*np.pi)) * (
            (y - eta)*np.sqrt( x*x + xi*xi ) * np.arctan(K) - (1./4.)*(x*x +xi*xi)*(1-K*K)
            * np.log((x*x + xi*xi)*(1+K*K))
            )
    return res

def getThFij_perp_x(r1, r2):
    # http://www.thermalradiation.net/sectionc/C-15.html
    if r1.cmin[0] == r2.cmin[2]:
        print("common edge case")
        return 0.
    x = np.array([r1.cmin[0], r1.cmax[0]])
    y = np.array([r1.cmin[1], r1.cmax[1]])
    xi = np.array([r2.cmin[2], r2.cmax[2]])
    eta = np.array([r2.cmin[1], r2.cmax[1]])

    f12 = 0.
    for l in range(1,3):
        for k in range(1,3):
            for j in range(1,3):
                for i in range(1,3):
                    f12 += np.power(-1, i+j+k+l)*g_perp_x(x[i-1], y[j-1], eta[k-1], xi[l-1])
    f12 *= 1./ ( (x[2-1] - x[1-1])*(y[2-1] - y[1-1]) )
    print(f"Th Fij perp ({r1.idt} -> {r2.idt}) = {f12}")

print("class Mesh: OK")

class Mesh: OK


## La Scène

- 'c' représente la hauteur "sous-plafond"

### Les normales

- on définit les normales qui seront utilisées par la suite (note : avec 3 points par face, on aurait pu les trouver automatiquement...)

In [9]:
nz_up = np.array([0., 0., 1.])
nz_down = np.array([0., 0., -1.])

ny_up = np.array([0., 1., 0.])
ny_down = np.array([0., -1., 0.])

nx_up = np.array([1., 0., 0.])
nx_down = np.array([-1., 0., 0.])

print("normal vectors: OK")

normal vectors: OK


### Construction de la scène

In [17]:
WIDTH = 3
HEIGHT = 3

c = 2.

mesh = Mesh()

## 1. Floor
print("## FLOOR ##")
for y in range(HEIGHT):
    for x in range(WIDTH):
        mesh.add(Rect(np.array([x, y, 0.]), np.array([x+1., y+1., 0.]), nz_up))

## 1. Ceiling
print("## CEILING ##")
for y in range(HEIGHT):
    for x in range(WIDTH):
        mesh.add(Rect(np.array([x, y, c]), np.array([x+1., y+1., c]), nz_down))

# North Wall
print("## NORTH WALL ##")
y = 0
for x in range(WIDTH):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x+1, y, c]), ny_up))

# South Wall
print("## SOUTH WALL ##")
y = HEIGHT
for x in range(WIDTH):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x+1, y, c]), ny_down))

# West Wall
print("## WEST WALL ##")
x = 0
for y in range(HEIGHT):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x, y+1, c]), nx_up))

# East Wall
print("## EAST WALL ##")
x = WIDTH
for y in range(HEIGHT):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x, y+1, c]), nx_down))

print("mesh: OK")

## FLOOR ##
0 [0. 0. 0.] [1. 1. 0.] area:1.0
1 [1. 0. 0.] [2. 1. 0.] area:1.0
2 [2. 0. 0.] [3. 1. 0.] area:1.0
3 [0. 1. 0.] [1. 2. 0.] area:1.0
4 [1. 1. 0.] [2. 2. 0.] area:1.0
5 [2. 1. 0.] [3. 2. 0.] area:1.0
6 [0. 2. 0.] [1. 3. 0.] area:1.0
7 [1. 2. 0.] [2. 3. 0.] area:1.0
8 [2. 2. 0.] [3. 3. 0.] area:1.0
## CEILING ##
9 [0. 0. 2.] [1. 1. 2.] area:1.0
10 [1. 0. 2.] [2. 1. 2.] area:1.0
11 [2. 0. 2.] [3. 1. 2.] area:1.0
12 [0. 1. 2.] [1. 2. 2.] area:1.0
13 [1. 1. 2.] [2. 2. 2.] area:1.0
14 [2. 1. 2.] [3. 2. 2.] area:1.0
15 [0. 2. 2.] [1. 3. 2.] area:1.0
16 [1. 2. 2.] [2. 3. 2.] area:1.0
17 [2. 2. 2.] [3. 3. 2.] area:1.0
## NORTH WALL ##
18 [0. 0. 0.] [1. 0. 2.] area:2.0
19 [1. 0. 0.] [2. 0. 2.] area:2.0
20 [2. 0. 0.] [3. 0. 2.] area:2.0
## SOUTH WALL ##
21 [0. 3. 0.] [1. 3. 2.] area:2.0
22 [1. 3. 0.] [2. 3. 2.] area:2.0
23 [2. 3. 0.] [3. 3. 2.] area:2.0
## WEST WALL ##
24 [0. 0. 0.] [0. 1. 2.] area:2.0
25 [0. 1. 0.] [0. 2. 2.] area:2.0
26 [0. 2. 0.] [0. 3. 2.] area:2.0
## EAST WALL ##


## Calcul des facteurs de forme (carrés)

In [20]:
# floor patch to ceiling patch (parallel, directly opposed)
th_fij = getThFij(1, 1, c)
# floor patch to wall patch (share an edge)
th_perp_x_fij = getThPerpFij(c, 1, 1)

theoritical para. (1,1,2.0) Fij = 0.0685895888185526
theoritical perp. (2.0,1,1) Fij = 0.2328526027953619


In [23]:
mesh.initFormFactors()
mesh.getFormFactors_uni_sq()
# Rect 1 : (1., 0, 0) -> (2, 1., 0)
mesh.showFij(1)

[1 -> 0] Fij = 0.0
[1 -> 1] Fij = 0.0
[1 -> 2] Fij = 0.0
[1 -> 3] Fij = 0.0
[1 -> 4] Fij = 0.0
[1 -> 5] Fij = 0.0
[1 -> 6] Fij = 0.0
[1 -> 7] Fij = 0.0
[1 -> 8] Fij = 0.0
[1 -> 9] Fij = 0.048026038407756225
[1 -> 10] Fij = 0.06954974267496243
[1 -> 11] Fij = 0.048026038407756225
[1 -> 12] Fij = 0.03476660478400199
[1 -> 13] Fij = 0.04802603840775622
[1 -> 14] Fij = 0.03476660478400198
[1 -> 15] Fij = 0.01613260431412167
[1 -> 16] Fij = 0.0202371320127766
[1 -> 17] Fij = 0.016132604314121668
[1 -> 18] Fij = 0.054503914644734544
[1 -> 19] Fij = 0.2240578913850361
[1 -> 20] Fij = 0.054503914644734544
[1 -> 21] Fij = 0.019716162254953883
[1 -> 22] Fij = 0.024791672262170248
[1 -> 23] Fij = 0.01971616225495389
[1 -> 24] Fij = 0.06568626051017326
[1 -> 25] Fij = 0.04150686708646163
[1 -> 26] Fij = 0.01565637571494623
[1 -> 27] Fij = 0.06568626051017325
[1 -> 28] Fij = 0.04150686708646162
[1 -> 29] Fij = 0.01565637571494623


## Constatations (partie 1)

Le patch 1 correspond à la surface du sol entre (1, 0, 0) à (2, 1, 0) dans la première rangée.

Points importants :

- 1 vers 10 (vers patch plafond immédiatement opposé) : 0.06954974267496243 vs 0.0685895888185526 soit 1.39% d'erreur
- 1 vers 19 (vers mur attenant au nord) : 0.2240578913850361 vs 0.2328526027953619 soit -3.77 % d'erreur

- les résultats montrent la symétrie attendue ce qui n'est pas le cas avec la méthode de lancé de rayons sur positions aléatoires (cf. plus loin)
    - 1 vers 9 (patch plafond à gauche) et 1 vers 11 (patch plafon à droite)
    - 1 vers 24 et 27 (les murs opposés de la première rangée)

In [24]:
getThFij_para_z(mesh.get(1), mesh.get(12))
getThFij_para_z(mesh.get(1), mesh.get(13))
getThFij_para_z(mesh.get(1), mesh.get(14))

getThFij_perp_x(mesh.get(1), mesh.get(24))
getThFij_perp_x(mesh.get(1), mesh.get(25))
getThFij_perp_x(mesh.get(1), mesh.get(26))

Th Fij (1 -> 12) = 0.035107100909694156
Th Fij (1 -> 13) = 0.04806410298507047
Th Fij (1 -> 14) = 0.035107100909694156
Th Fij perp (1 -> 24) = 0.06574698897816222
Th Fij perp (1 -> 25) = 0.041967204961217586
Th Fij perp (1 -> 26) = 0.01591710403613758


## Constatations (partie 2)

(**TODO:** graphique erreur sur les cas où on dispose des résultats théoriques)

## Calcul des facteurs de forme (aléatoire)

In [30]:
mesh.initFormFactors()
mesh.getFormFactors_rnd(16)
# Rect 1 : (1., 0, 0) -> (2, 1., 0)
mesh.showFij(1)

[1 -> 0] Fij = 0.0
[1 -> 1] Fij = 0.0
[1 -> 2] Fij = 0.0
[1 -> 3] Fij = 0.0
[1 -> 4] Fij = 0.0
[1 -> 5] Fij = 0.0
[1 -> 6] Fij = 0.0
[1 -> 7] Fij = 0.0
[1 -> 8] Fij = 0.0
[1 -> 9] Fij = 0.04705024357793468
[1 -> 10] Fij = 0.06867592957482017
[1 -> 11] Fij = 0.05192578072548309
[1 -> 12] Fij = 0.0326934976144591
[1 -> 13] Fij = 0.048505758197937836
[1 -> 14] Fij = 0.03468846343353401
[1 -> 15] Fij = 0.022775598910145804
[1 -> 16] Fij = 0.01880027133577495
[1 -> 17] Fij = 0.015711832649565417
[1 -> 18] Fij = 0.06656328508288425
[1 -> 19] Fij = 0.17222668654866718
[1 -> 20] Fij = 0.06405873504469553
[1 -> 21] Fij = 0.018403634321533035
[1 -> 22] Fij = 0.024327767573218397
[1 -> 23] Fij = 0.01879774161696582
[1 -> 24] Fij = 0.0541948282030384
[1 -> 25] Fij = 0.03623454528679388
[1 -> 26] Fij = 0.016675844492583494
[1 -> 27] Fij = 0.06504529381035953
[1 -> 28] Fij = 0.03649221324150836
[1 -> 29] Fij = 0.016895792857653066


- adieu la symétrie... à moins de détecter les cas et forcer les valeurs (moyenne, remplacement ?)
- avec n faible, l'erreur est plus importante
- le méthode est plus longue que celle basée sur la subdivision uniforme (mais le résultat de cette dernière est sans doute plus grossier dans le cadre d'une utilisation "normale"... ce qui n'est pas le cas ici

## Et maintenant ?

Prochaine étape : résolution des systèmes d'équations pour le calcul de radiosité